In [1]:
%reload_ext autoreload
%autoreload 2
# BASE
import os
import json
# COOL
from pyeed.core import ProteinRecord
from pyeed.core import DNARecord

In [2]:
# read in the protein records from the json
current_path = os.path.dirname(os.getcwd())
read_blast_folder = os.path.join(current_path, "TEM-lactamase", "data", "data_fetch_list_ids_protein_dna_all")
# /home/niklas/Desktop/Job_Niklas/TEM-lactamase/data/data_blast_search_big_list001test_all_10
# list all the files in the directory
# we read in the proteins and ids from input folder
protein_dic = {}

for file in os.listdir(read_blast_folder):
    if file.endswith(".json"):
        with open(os.path.join(read_blast_folder, file), "r") as f:
            # read in the file as a dic
            file_data = json.load(f)
            protein_dic[file.split('.')[0]] = {'protein': ProteinRecord.from_json_string(file_data['protein']), 'dna': DNARecord.from_json_string(file_data['dna'])}

print(len(protein_dic))


print('The fetched list of results is: ' +  str(len(protein_dic)))

209
The fetched list of results is: 209


In [3]:
# read in the file from json
tem_dic = {}

with open(os.path.join(current_path, "TEM-lactamase", "TEM_Ids.json"), "r") as f:
    tem_dic = json.load(f)

In [4]:
# now we run clustal omega on the proteins
from pyeed.tools.clustalo import ClustalOmega

sequences = [protein['protein'].sequence for protein in protein_dic.values()]

# fix the format for clustal omega
# sequences = [
#     ">seq1\nMTHKLLLTLLFTLLFSSAYSRG",
#     ">seq2\nMTHKILLLTLLFTLLFSSAYSRG",
#     ">seq3\nMTHKILLLTLLFTLLFSSCYSRG",
# ]
sequences_filtered = []

for i, seq in enumerate(sequences):
    sequences_filtered.append(">seq" + str(i) + "\n" + seq)


clustalo = ClustalOmega()
alignment = clustalo.align(sequences_filtered)

# now we need the alignment put in the dic for later use and export
for i, protein in enumerate(protein_dic.values()):
    protein['alignment'] = str(alignment[i].seq)


In [5]:
output_folder_blast_search = os.path.join(current_path, "TEM-lactamase", "data", "alignment_protein_ids_fetch_all")
os.makedirs(output_folder_blast_search, exist_ok=True)

def dumper(obj):
    try:
        return obj.json()
    except:
        return obj.__dict__

# now we save the dic with the alignment in json format
# save the blast search results
counter = 0
for key, value in protein_dic.items():
    with open(os.path.join(output_folder_blast_search, key + ".json"), "w") as f:
        json.dump(value, f, default=dumper)
        counter += 1